In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
tweets_df = pd.read_csv('/content/gdrive/MyDrive/sentiment_analysis/training.1600000.processed.noemoticon.csv',encoding='latin-1',names=['target','ids','date','flag','username','text'])
tweets_df.head()

,target,ids,date,flag,username,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
tweets_df.columns

Index(['target', 'ids', 'date', 'flag', 'username', 'text'], dtype='object')

In [ ]:
X = list(tweets_df['text'])
Y = list(tweets_df['target'])
print("X : ",len(X),"\nY : ", len(Y))

X :  1600000 
Y :  1600000


In [ ]:
tweets_df.describe()

,target,ids
count,1.600000e+06,1.600000e+06
mean,2.000000e+00,1.998818e+09
std,2.000001e+00,1.935761e+08
min,0.000000e+00,1.467810e+09
25%,0.000000e+00,1.956916e+09
50%,2.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state = 0)
X_train = X_train[:32000]
Y_train = Y_train[:32000]
X_test = X_test[:80000]
Y_test = Y_test[:80000]
print("X_train : ",len(X_train),"\nX_test : ",len(X_test))

X_train :  32000 
X_test :  80000


In [ ]:
count1 = 0
count2 = 0
count3 = 0
for i in Y_test:
  if i==0:
    count1+=1
  elif i==2:
    count2+=1
  else:
    count3+=1

print("Negative:",count1,"\nNeutral:",count2,"\nPositive:",count3)

Negative: 39989 
Neutral: 0 
Positive: 40011


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.4 MB/s 
     |████████████████████████████████| 6.6 MB 46.4 MB/s 
     |████████████████████████████████| 101 kB 12.5 MB/s 
     |████████████████████████████████| 596 kB 63.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import DistilBertTokenizerFast
import tensorflow as tf

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [ ]:
X_train_encodings = tokenizer(X_train, truncation = True, padding = True)
X_test_encodings = tokenizer(X_test, truncation = True, padding = True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encodings),
    Y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test_encodings),
    Y_test
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='/content/gdrive/MyDrive/sentiment_analysis',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    eval_steps=10,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/gdrive/MyDrive/sentiment_analysis',            # directory for storing logs
    logging_steps=10,
)

In [ ]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_39', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
trainer.save_model('twitter_sentiment')

In [ ]:
model.save_pretrained('/content/gdrive/MyDrive/sentiment_analysis2',saved_model=True)

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/sentiment_analysis2/saved_model/1/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/sentiment_analysis2/saved_model/1/assets


In [ ]:
trainer.evaluate(test_dataset)

{'eval_loss': nan}

In [ ]:
output=trainer.predict(test_dataset)[1]

In [ ]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(Y_test,output)
cm

array([[39989,     0],
       [    0, 40011]])